In [ ]:
import gradio as gr
import numpy as np
import cv2
import joblib
from skimage.feature import local_binary_pattern as lbp
from skimage import exposure

# Function to compute colour histogram features for an image
def computeColourHist(image, bins=20):
    hsvImg = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hsvImg = hsvImg / 255.0
    hueHist = np.histogram(hsvImg[:,:,0], bins=bins, range=(0, 1))[0]
    satHist = np.histogram(hsvImg[:,:,1], bins=bins, range=(0, 1))[0]
    valHist = np.histogram(hsvImg[:,:,2], bins=bins, range=(0, 1))[0]
    return np.concatenate((hueHist, satHist, valHist))

def computeLBP_Features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    lbpImg = lbp(gray, method="uniform", P=8, R=1)
    bins = np.arange(0, 11)
    lbpHist, _ = np.histogram(lbpImg.flatten(), bins=bins, range=(0, 10))
    return lbpHist / (np.sum(lbpHist) + 1e-6)

def extract_features(image):
    image = cv2.resize(image, (100, 100))  # resize to match training input
    ch_features = computeColourHist(image)
    lbp_features = computeLBP_Features(image)
    return np.concatenate([ch_features, lbp_features])

# Load the trained model
model = joblib.load("random_forest_model.pkl")
scaler = joblib.load('scaler.pkl')

def predict(image):
    features = extract_features(image).reshape(1, -1)
    scaled_features = scaler.transform(features) # Scale the features
    prediction = model.predict(scaled_features)[0]
    return f"Predicted Fruit: {prediction}"

# Gradio Interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="numpy"),
    outputs="text",
    title="Fruit Classifier (LBP + Color Histogram)",
    description="Upload an image of a fruit to classify it using a Random Forest model."
)

interface.launch()